# Logistic Regression approach 

In [1]:
import numpy as np 
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('pairwise_data_test200.csv')
print(df)

       Unnamed: 0  Bedrooms1  Bathrooms1   Size1  Crime1  Transit1  Shopping1  \
0               0        2.0         1.0   699.0     6.3      90.1       90.1   
1               1        2.0         1.0   699.0     1.4       3.5       22.5   
2               2        2.0         1.0   699.0     1.4       3.5       22.5   
3               3        2.0         2.0  1199.0    33.8      50.7       27.5   
4               4        2.0         2.0  1199.0    33.8      50.7       27.5   
5               5        2.0         2.0  1199.0    33.8      50.7       27.5   
6               6        3.0         2.0  1100.0    18.3      29.6       59.9   
7               7        3.0         2.0  1100.0    18.3      29.6       59.9   
8               8        3.0         2.0  1100.0    18.3      29.6       59.9   
9               9        3.0         2.0  1100.0    18.3      29.6       59.9   
10             10        2.0         2.0  1500.0    38.7      20.4       38.7   
11             11        2.0

In [3]:
df = df.drop('Unnamed: 0', 1)
print(df)

       Bedrooms1  Bathrooms1   Size1  Crime1  Transit1  Shopping1  Health1  \
0            2.0         1.0   699.0     6.3      90.1       90.1     45.8   
1            2.0         1.0   699.0     1.4       3.5       22.5     17.6   
2            2.0         1.0   699.0     1.4       3.5       22.5     17.6   
3            2.0         2.0  1199.0    33.8      50.7       27.5     10.6   
4            2.0         2.0  1199.0    33.8      50.7       27.5     10.6   
5            2.0         2.0  1199.0    33.8      50.7       27.5     10.6   
6            3.0         2.0  1100.0    18.3      29.6       59.9     14.8   
7            3.0         2.0  1100.0    18.3      29.6       59.9     14.8   
8            3.0         2.0  1100.0    18.3      29.6       59.9     14.8   
9            3.0         2.0  1100.0    18.3      29.6       59.9     14.8   
10           2.0         2.0  1500.0    38.7      20.4       38.7     47.2   
11           2.0         2.0  1500.0    38.7      20.4       38.

In [62]:
matrix_df = df.as_matrix()

matrix_df = matrix_df/np.amax(matrix_df, axis=0)

print(matrix_df.shape)
print(matrix_df)

(19900, 25)
[[ 0.5         0.2         0.16158114 ...,  0.09330629  0.15720081  1.        ]
 [ 0.5         0.2         0.16158114 ...,  0.09330629  0.15720081  0.        ]
 [ 0.5         0.2         0.16158114 ...,  0.84279919  0.94320487  0.        ]
 ..., 
 [ 0.5         0.2         0.16158114 ...,  0.23529412  0.65720081  1.        ]
 [ 0.5         0.2         0.16158114 ...,  0.52129817  1.          0.        ]
 [ 0.5         0.2         0.16158114 ...,  0.52129817  1.          0.        ]]


In [5]:
def next_batch(mat, index, size):
    if index + size <= mat.shape[0]:
        return index+size, mat[index:index+size]
    else:
        return size, mat[:size]

def expand_label(labels):
    return np.array([[0,1] if label == 1 else [1,0] for label in labels])

In [6]:
y_new = expand_label(matrix_df[:,-1])
print(y_new[:10])

[[0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


In [100]:
learning_rate = 0.0001
training_epochs = 200
batch_size = 100
display_step = 50
test_num = 100
total_batch = int(matrix_df.shape[0]/batch_size)
print(total_batch)

x = tf.placeholder(tf.float32, [None, 24]) 
y = tf.placeholder(tf.float32, [None, 2])

W = tf.Variable(tf.truncated_normal(shape=(24, 128), dtype=tf.float32))
b = tf.Variable(tf.zeros([128]))

W2 = tf.Variable(tf.truncated_normal(shape=(128, 64), dtype=tf.float32))
b2 = tf.Variable(tf.zeros([64]))

W3 = tf.Variable(tf.truncated_normal(shape=(64, 2), dtype=tf.float32))
b3 = tf.Variable(tf.zeros([2]))

h2 = tf.nn.relu(tf.matmul(x, W) + b)
h3 = tf.nn.relu(tf.matmul(h2, W2) + b2)
h3_drop = tf.nn.dropout(h3, keep_prob=0.5)

pred = tf.nn.softmax(tf.nn.relu(tf.matmul(h3_drop, W3) + b3)) 
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, 1e-10, 1)), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()

199


In [ ]:
for i in range(10):
    with tf.Session() as sess:
        sess.run(init)

        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.0
            index = 0
            # Loop over all batches
            for i in range(total_batch):
                index, batch = next_batch(matrix_df, index, batch_size)
    #             print(index)
                batch_xs, batch_ys = (batch[:,:-1].astype(float), expand_label(batch[:,-1].astype(float)))
    #             print(batch_xs.shape)
    #             print(batch_ys.shape)
                # Fit training using batch data
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                              y: batch_ys})
                # Compute average loss
    #             print(c)
                avg_cost += c / total_batch
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

        print("Optimization Finished!")
        print(W3.eval())

        x_, y_ = (matrix_df[300:400,:-1], matrix_df[300:400,-1])
        prediction = sess.run(pred, feed_dict={x: x_, y: expand_label(y_)})
        print(prediction[:10])
        final_prediction = [1 if item[0] < item[1] else 0 for item in prediction]
        accuracy = abs(np.sum(final_prediction - y_))/test_num * 100
        print(final_prediction[:10])
        print("Accuracy: ", accuracy, "%")

Epoch: 0050 cost= 0.975417657
Epoch: 0100 cost= 0.773002101
Epoch: 0150 cost= 0.743647078
Epoch: 0200 cost= 0.715402147
Optimization Finished!
[[ 1.14704394  0.00963418]
 [-0.39372104 -0.15676147]
 [-1.33854961  0.67933565]
 [ 0.59925246  0.63261074]
 [ 1.85515964 -0.70839381]
 [-0.26726845 -0.7182653 ]
 [-0.01796398 -0.37074137]
 [-0.7109043  -0.57749701]
 [-0.90035552 -0.65419054]
 [ 0.21147163  0.66470122]
 [ 0.95208037  0.24545285]
 [ 0.01138274 -0.85563052]
 [ 0.40802714 -0.97969794]
 [-0.64613271  0.96342367]
 [ 1.47901011  0.72339666]
 [-0.19254433  0.1303155 ]
 [ 1.40825009  1.05881011]
 [-0.23175539  0.6127587 ]
 [-1.40597224 -0.16225833]
 [ 0.07684959  1.62919867]
 [-1.2966584  -0.56898695]
 [ 0.16849743 -0.37458411]
 [ 1.13557172  1.5378958 ]
 [-0.87475622 -0.05325478]
 [ 1.10707712  0.66373789]
 [-0.4446899   1.01243913]
 [ 0.68211389  0.42997345]
 [-0.82821381 -0.91551816]
 [-0.18460245  0.5167613 ]
 [ 0.9723084   0.4398104 ]
 [-0.43060058  1.80664909]
 [-1.5229156  -0.565